In [1]:
import os
import numpy as np
import pandas as pd
import mne
import matplotlib.pyplot as plt
from autoreject import AutoReject, Ransac
from autoreject.utils import interpolate_bads
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs, find_bad_channels_maxwell
from IPython.display import display, HTML
%matplotlib qt

In [2]:
montage = mne.channels.make_standard_montage("GSN-HydroCel-128")

In [14]:
raw = mne.io.read_raw_egi("C:/Learn/Project/bylw/eeg/1/lal-hc-405-task_20230825_084820.mff", preload=True)
raw_copy = raw.copy()
raw_copy.set_montage(montage, on_missing='warn')
# raw_copy.plot()

Reading EGI MFF Header from C:\Learn\Project\bylw\eeg\1\lal-hc-405-task_20230825_084820.mff...
    Reading events ...
    Assembling measurement info ...
    Excluding events {} ...
Reading 0 ... 1637160  =      0.000 ...  3274.320 secs...


C:\Users\24323\AppData\Local\Temp\ipykernel_44160\2895094055.py:3: RuntimeWarning: DigMontage is only a subset of info. There is 1 channel position not present in the DigMontage. The channel missing from the montage is:

['VREF'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if this is not an EEG channel, or use the on_missing parameter if the channel position is allowed to be unknown in your analyses.
  raw_copy.set_montage(montage, on_missing='warn')


<RawMff | signal1.bin, 155 x 1637161 (3274.3 s), ~1.89 GiB, data loaded>

In [4]:
# fig = raw_copy.compute_psd().plot(
#     average=True, amplitude=False, picks="data", exclude="bads"
# )

In [15]:
events, event_dict = mne.events_from_annotations(raw_copy)

# 统计每个事件的数量
for event_name, event_code in event_dict.items():
    count = sum(events[:, 2] == event_code)
    print(f"事件 {event_name} (ID={event_code}) 出现次数: {count}")

Used Annotations descriptions: ['100 ', '101 ', '102 ', '103 ', '104 ', '105 ', '106 ', '107 ', '108 ', '109 ', '110 ', '111 ', '112 ', '113 ', '114 ', '115 ', '2   ', '255 ', '3   ', '4   ', '5   ', '6   ', '800 ', '801 ', '900 ', '901 ']
事件 100  (ID=1) 出现次数: 64
事件 101  (ID=2) 出现次数: 69
事件 102  (ID=3) 出现次数: 39
事件 103  (ID=4) 出现次数: 35
事件 104  (ID=5) 出现次数: 13
事件 105  (ID=6) 出现次数: 15
事件 106  (ID=7) 出现次数: 3
事件 107  (ID=8) 出现次数: 1
事件 108  (ID=9) 出现次数: 56
事件 109  (ID=10) 出现次数: 54
事件 110  (ID=11) 出现次数: 51
事件 111  (ID=12) 出现次数: 43
事件 112  (ID=13) 出现次数: 12
事件 113  (ID=14) 出现次数: 20
事件 114  (ID=15) 出现次数: 1
事件 115  (ID=16) 出现次数: 2
事件 2    (ID=17) 出现次数: 480
事件 255  (ID=18) 出现次数: 2
事件 3    (ID=19) 出现次数: 480
事件 4    (ID=20) 出现次数: 480
事件 5    (ID=21) 出现次数: 480
事件 6    (ID=22) 出现次数: 480
事件 800  (ID=23) 出现次数: 205
事件 801  (ID=24) 出现次数: 35
事件 900  (ID=25) 出现次数: 207
事件 901  (ID=26) 出现次数: 33


In [6]:
target_event_ids = list(range(1, 17))
# 遍历修改
for i in range(len(events)):
    if events[i, 2] in target_event_ids:
        events[i, 2] = 1  # 修改事件 ID 为 1
# 显示修改后的结果
print("修改后的 events：")
events

修改后的 events：


array([[   3903,       0,      17],
       [   4149,       0,      19],
       [   4151,       0,      23],
       ...,
       [1631664,       0,      21],
       [1631914,       0,      22],
       [1632166,       0,       1]])

In [16]:
# 去除channel
raw_copy.drop_channels(['E1', 'E2','E9','E14', 'E15', 'E17', 'E21','E22', 'E26', 'E32', 'E38', 'E39', 'E43', 'E44', 'E45', 'E48', 'E49', 'E108', 'E113', 'E114', 'E115', 'E119', 'E120', 'E121', 'E125', 'E128'])
# raw_copy.ch_names
eogs = {item:'eog' for item in [ 'E8', 'E25', 'E126', 'E127']}
raw_copy.set_channel_types({'VREF': 'misc'})
raw_copy.set_channel_types(eogs)

C:\Users\24323\AppData\Local\Temp\ipykernel_44160\1089640693.py:5: RuntimeWarning: The unit for channel(s) VREF has changed from V to NA.
  raw_copy.set_channel_types({'VREF': 'misc'})


<RawMff | signal1.bin, 129 x 1637161 (3274.3 s), ~1.57 GiB, data loaded>

In [8]:
raw_copy.set_montage(montage)
# fig = raw_copy.plot_sensors(show_names=True)

C:\Users\24323\AppData\Local\Temp\ipykernel_44160\3161625792.py:1: RuntimeWarning: Not setting positions of 4 eog channels found in montage:
['E8', 'E25', 'E126', 'E127']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw_copy.set_montage(montage)


<RawMff | signal1.bin, 129 x 1637161 (3274.3 s), ~1.57 GiB, data loaded>

In [9]:
# 重参考
raw_copy.set_eeg_reference('average', ch_type='eeg')

Applying average reference.
Applying a custom ('EEG',) reference.


<RawMff | signal1.bin, 129 x 1637161 (3274.3 s), ~1.57 GiB, data loaded>

In [ ]:
raw_copy.plot_psd()

In [10]:
# 滤波
picks = mne.pick_types(raw_copy.info, eeg=True, eog=True)
raw_copy.filter(l_freq=0.1, h_freq=30, fir_design='firwin', phase='zero-double', picks=picks)


Filtering raw data in 4 contiguous segments
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-12 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-12 dB cutoff frequency: 33.75 Hz)
- Filter length: 16501 samples (33.002 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 102 out of 102 | elapsed:    3.5s finished


<RawMff | signal1.bin, 129 x 1637161 (3274.3 s), ~1.57 GiB, data loaded>

In [11]:
# fig = raw_copy.compute_psd().plot(
#     average=True, amplitude=False, picks="data", exclude="bads"
# )

In [12]:

# raw_copy.resample(200)

In [13]:
tmin_1, tmax_1 = -0.7, 1     # Mark=1 的时间范围

# 提取 Mark=1 的 epochs
epochs_combined = mne.Epochs(raw_copy, events, event_id=19, 
                       tmin=tmin_1, tmax=tmax_1, 
                       baseline=None, preload=True)

# ica
ica = mne.preprocessing.ICA(n_components=15, random_state=97, method="infomax", max_iter=3000)
ica.fit(epochs_combined)

eog_inds, scores = ica.find_bads_eog(epochs_combined,measure='correlation',threshold=0.4)
ica.exclude = eog_inds
print(eog_inds)

epochs_combined = ica.apply(epochs_combined.copy())

Not setting metadata
480 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 480 events and 851 original time points ...
0 bad epochs dropped
Fitting ICA to data using 98 channels (please be patient, this may take a while)
Selecting by number: 15 components
Computing Infomax ICA
Fitting ICA took 117.2s.
Using EOG channels: E8, E25, E126, E127
[0, 7, 2, 6, 1]
Applying ICA to Epochs instance
    Transforming to ICA space (15 components)
    Zeroing out 5 ICA components
    Projecting back using 98 PCA components


In [ ]:
# 截取

# 时间窗口
tmin_1, tmax_1 = -0.2, 0     # Mark=1 的时间范围
tmin_2, tmax_2 = 0, 1.5   # Mark=2 的时间范围

# 提取 Mark=1 的 epochs
epochs_1 = mne.Epochs(raw_copy, events, event_id=1, 
                       tmin=tmin_1, tmax=tmax_1, 
                       baseline=None, preload=True)

# 提取 Mark=2 的 epochs
epochs_2 = mne.Epochs(raw_copy, events, event_id=17, 
                       tmin=tmin_2, tmax=tmax_2, 
                       baseline=None, preload=True)
epochs_2.drop(indices=[0, 1])  # 删除第 0 和第 1 个 epoch

# # ica
# ica = mne.preprocessing.ICA(n_components=0.99, random_state=97, method="infomax", max_iter=3000)
# ica.fit(epochs_2)
# ica.plot_components()

# eog_inds, scores = ica.find_bads_eog(epochs_2,measure='correlation',threshold=0.4)
# ica.exclude = eog_inds
# print(eog_inds)

# epochs_2 = ica.apply(epochs_2.copy())

# 确保两个 epochs 数量相等
assert len(epochs_1) == len(epochs_2), "Mark=1 和 Mark=2 事件数量不匹配！"

trial_1 = epochs_1.get_data()  # 取 Mark=1 片段
trial_2 = epochs_2.get_data()  # 取 Mark=2 片段
print("trial2_shape: ", trial_2.shape)
    
# 在时间轴 (axis=2) 方向拼接
concatenated_data = np.concatenate([trial_1, trial_2], axis=2)

# 转换为 NumPy 数组
concatenated_data = np.array(concatenated_data)

# 创建新的 EpochsArray
info = raw_copy.info  # 继承原始数据的通道信息
epochs_combined = mne.EpochsArray(concatenated_data, info)
# epochs_combined.filter(None,30,fir_design='firwin', phase='zero-double', picks=picks)
# 查看新 epochs 结构
print(epochs_combined)



In [ ]:
# ica
ica = mne.preprocessing.ICA(n_components=0.9, random_state=97, method="infomax", max_iter=3000)
ica.fit(epochs_combined)

eog_inds, scores = ica.find_bads_eog(epochs_combined,measure='correlation',threshold=0.4)
ica.exclude = eog_inds
print(eog_inds)

epochs_combined = ica.apply(epochs_combined.copy())

In [ ]:
ica.plot_sources(epochs_combined)
ica.plot_components()

In [ ]:
# pick_ica = [0,1,2,3]
# ica.plot_properties(epochs_combined)


In [ ]:
# pick_ica = [11]
# ica.plot_properties(epochs_combined, picks=pick_ica)

In [ ]:
epochs_combined

In [ ]:
epochs_combined.apply_baseline((-0.7,-0.5))

evoke = epochs_combined.average()
evoke.pick(["E6", "E62"])
evoke.plot_joint() 

In [ ]:
# 自动删除坏导
# 初始化AutoReject，可自定义参数（如n_interpolate控制最大插值通道数）
picks = mne.pick_types(epochs_combined.info,eeg=True, eog=False)
ar = Ransac(verbose=True, picks=picks, n_jobs=1, min_channels=0.1)
epochs_clean = ar.fit_transform(epochs_combined)
# epochs_clean.apply_baseline((-0.2,0))

In [ ]:
epochs_clean.average().plot_joint()

In [ ]:
epochs.events

In [ ]:
raw.info['nchan']

In [ ]:
raw.info